In [1]:
from IPython.display import HTML
from IPython.display import Image
import warnings
warnings.filterwarnings("ignore")

<img src='interview.jpg' width="700" height="300" align="center">

# Examining Racial Discrimination in the US Job Market


In [2]:
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')


### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

Import packages

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats 
from scipy.stats import truncnorm
#import statsmodels.api as sm
import statsmodels.stats.api as sm
import pylab
from statsmodels.stats.weightstats import ztest
from IPython.display import display, Math, Latex
#from numpy.random import randn
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

In [26]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [27]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [28]:
data.describe()

,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,...,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind
count,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,...,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000
mean,3.618480,3.661396,7.842916,0.052772,0.411499,0.097125,0.448049,215.637782,3.481520,0.559548,...,0.106776,0.437166,0.072690,0.082957,0.030390,0.085010,0.213963,0.267762,0.154825,0.165092
std,0.714997,1.219126,5.044612,0.223601,0.492156,0.296159,0.497345,148.127551,2.038036,0.496492,...,0.308866,0.496083,0.259649,0.275854,0.171677,0.278932,0.410141,0.442847,0.361773,0.371308
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,27.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,267.000000,4.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,4.000000,9.000000,0.000000,1.000000,0.000000,1.000000,313.000000,6.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,4.000000,7.000000,44.000000,1.000000,1.000000,1.000000,1.000000,903.000000,6.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

What test is appropriate for this problem? Does CLT apply?

In [30]:
w=data.race[data.race=='w']
b=data.race[data.race=='b']
calls_w=data.call[data.race=='w']
calls_b=data.call[data.race=='b']
print('Number of black sounding names:',b.count(),'Return calls:',calls_w.sum(),'\nNumber of white sounding names:',w.count(),'Return calls:',calls_b.sum(),)

Number of black sounding names: 2435 Return calls: 235.0 
Number of white sounding names: 2435 Return calls: 157.0


- There are far more than 30 observations.
- *Note: We are dealing with something more like categorical data (Bernoulli distribution) as opposed to continuous data so the testing approach will be different.*


In [31]:
c1,n1,c2,n2=157,2435,235,2435
p1=c1/n1
p2=c2/n2
print('Black sounding names that recieved a call back: ',p1)
print('White sounding names that recieved a call back: ',p2)


Black sounding names that recieved a call back:  0.06447638603696099
White sounding names that recieved a call back:  0.09650924024640657


What are the null and alternate hypotheses?

### Testing Assumptions

**Random**

Since we are told that the researchers randomly assigned the résumés, then we have to automatically assume that our data is random by default.

        "Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers."

**Normality**

In [32]:
Math(r'n * \hat p_{success} , n(1-\hat p_{success}) \geq 10 ')

<IPython.core.display.Math object>

In [33]:
n=0
if n1*(p1)>=10:
    print('normal (2435*(157/2435))')
    n+=1
if n1*(1-p1)>=10:
    print('normal (2435*(1-157/2435))')
    n+=1
if n2*(p2)>=10:
    print('normal (2435*(235/2435))')
    n+=1
if n2*(1-p2)>=10:
    print('normal (2435*(1-235/2435))')
    n+=1
if n==4:
    print('All tests passed, the data is normal!')


normal (2435*(157/2435))
normal (2435*(1-157/2435))
normal (2435*(235/2435))
normal (2435*(1-235/2435))
All tests passed, the data is normal!


**Independance**

Is the sample size <10% of the population?

*We assume that 2435 résumés are well below the population number of résumés in the United States.*

**Hypothesis Test**

In [34]:
print('If we fail to reject the null hypothesis then the data would seem consistent with the claim that the proportions of white and black sounding names receiving a call back is equivalent. ')
Math(r'H_{0} : p_{w} - p_{b} = 0 \\ H_{1} : p_{w} - p_{b} \neq 0')

If we fail to reject the null hypothesis then the data would seem consistent with the claim that the proportions of white and black sounding names receiving a call back is equivalent. 


<IPython.core.display.Math object>

In [35]:
Math(r'z: = \dfrac{(p_1 - p_2)}{\sqrt{\dfrac{p_1*(1-p_1)}{n_1} + \dfrac{p_2*(1-p_2)}{n_2} }}')

<IPython.core.display.Math object>

In [36]:
z=1.96

v1=(p1*(1-p1)/n1)
v2=(p2*(1-p2)/n2)
se=np.sqrt((v1)+(v2))

ztest=(p1-p2)/se
upper_bound=(p1-p2)+1.96*se
lower_bound=(p1-p2)-1.96*se
print('Estimate:',ztest, '\n95% C.I.',(lower_bound,upper_bound))

Estimate: -4.11555043573 
95% C.I. (-0.047288260559332024, -0.016777447859559147)


In [37]:
print('The margin of error:',1.96*se)

The margin of error: 0.015255406349886438


In [38]:
stat,pval = stats.ttest_ind(calls_b, calls_w)
ci = sm.CompareMeans(sm.DescrStatsW(calls_b), sm.DescrStatsW(calls_w)).tconfint_diff(usevar='unequal')
print('Estimate:',stat,'\np-value:',pval)
print('95% C.I. ',ci)

Estimate: -4.114705290861751 
p-value: 3.940802103128886e-05
95% C.I.  (-0.04729503443489937, -0.016770673983991798)


In [39]:
from statsmodels.stats.proportion import proportions_ztest

count =(c1,c2)
nobs =(n1,n2)
value=0
zstat, pval = proportions_ztest(count, nobs, value=value, alternative='two-sided')
print('Estimate:',zstat,'\np-value:',pval)

Estimate: -4.108412152434346 
p-value: 3.983886837585077e-05


#### 2x2 Chi-Square Contingency Tables

In [40]:
Math(r'X^2: = \dfrac{(O - E)^2}{E}')

<IPython.core.display.Math object>

In [41]:
#Frequency table
data_race_call=data[['call','race']]
data_race_call.call[data_race_call['call']==0]=2
data_race_call.call[data_race_call.race=='b'].value_counts()
data_race_call.call[data_race_call.race=='w'].value_counts()
call_table_freq=pd.DataFrame(data_race_call.call[data_race_call.race=='b'].value_counts())
call_table_freq['call2']=data_race_call.call[data_race_call.race=='w'].value_counts()
call_table_freq['totals']=call_table_freq['call']+call_table_freq['call2']
call_table_freq.sort_index(inplace=True)
call_table_freq.columns=['black','white','totals']
call_table_freq.loc[3]=[sum(call_table_freq.black),sum(call_table_freq.white),sum(call_table_freq.totals)] 
call_table_freq.index=['call','no call','totals']
#Probability table
call_table_prob=call_table_freq/4870
call_table_freq=call_table_freq.T
call_table_prob=call_table_prob.T
display(call_table_freq) 
display(call_table_prob)


,call,no call,totals
black,157,2278,2435
white,235,2200,2435
totals,392,4478,4870


,call,no call,totals
black,0.032238,0.467762,0.5
white,0.048255,0.451745,0.5
totals,0.080493,0.919507,1.0


In [42]:
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from scipy.stats import chi2_contingency,chi2
from scipy.stats.distributions import chi2
import scipy, scipy.stats
#from scipy.stats.distributions.chi2 import sf

In [43]:
chiSquareTemp=call_table_freq
#observed
a,b,c,d=chiSquareTemp.iloc[1,1],chiSquareTemp.iloc[1,0],chiSquareTemp.iloc[0,1],chiSquareTemp.iloc[0,0]
#Totals
col1Sum=np.sum(a+c)
col2Sum=np.sum(b+d)
row1Sum=np.sum(a+b)
row2Sum=np.sum(c+d)
totalSum=np.sum(col1Sum+col2Sum)

   #row total               column total
e=(col1Sum*row1Sum)/totalSum #a expected
f=(col2Sum*row1Sum)/totalSum #b expected

g=(col1Sum*row2Sum)/totalSum #c expected
h=(col2Sum*row2Sum)/totalSum #d expected
#Modify table labels
obs=pd.DataFrame([[a,b],[c,d]])

#Get Chi2 and LR for table
chi2, pvalue1, dof1, expected1 = chi2_contingency(obs,correction=True)
LR, pvalue2, dof2, expected2 = chi2_contingency(obs, lambda_="log-likelihood")
#calculated phi for table
phi=(a*d-b*c)/np.sqrt((a+b)*(c+d)*(a+c)*(b+d))
phiTable=list([phi,'',''])
estimates=pd.DataFrame([[chi2,dof1,pvalue1],[LR,dof2,pvalue2],phiTable],columns=['Value','DF','Prob'])
#Rename Indexes 
estimates.rename(index={0: 'Chi-Square',1: 'Likelihood Ratio Chi2',2:'Phi Coefficient'},inplace=True)
estimates

,Value,DF,Prob
Chi-Square,16.449029,1,4.99758e-05
Likelihood Ratio Chi2,16.547891,1,4.74367e-05
Phi Coefficient,-0.058872,,


In [44]:
table = np.asarray(obs)
results = sm.Table2x2(table.T)
print(results.summary())

               Estimate   SE   LCB    UCB   p-value
---------------------------------------------------
Odds ratio        0.645        0.523  0.796   0.000
Log odds ratio   -0.438 0.107 -0.649 -0.228   0.000
Risk ratio        0.820        0.752  0.893   0.000
Log risk ratio   -0.199 0.044 -0.285 -0.113   0.000
---------------------------------------------------


**Explanation**: Those résumés with black sounding names were 0.645 times less likely to receive a call back compared to white sounding names.

#### Variables of Interest

I minimized the dataset to variables of interest and recoded them based on referent categories. I selected white males to be referent categories because they are most likely favored for number of calls. After that I created dummy variables based on years of experience where years were divided to 3 distinct categories (0-5,5-9,10+).

In [45]:
#referent groups (education=0(3&4),yearsexp=0(>=10),male=0,white=0)
voi_data=data[['education','yearsexp','sex','race','call']]
voi_data['race']=voi_data.race.replace('b',1).replace('w',0)#b=1,w=0
voi_data['sex']=voi_data.sex.replace('m',0).replace('f',1)#m=0,f=1
#(3&4=0),(0,1,2=1)

bin1=voi_data.education<3
bin2=voi_data.education>2
voi_data.education[bin1]=0
voi_data.education[bin2]=1

voi_data['yearsxpcats']='.'

#yearsexp
bin1=voi_data.yearsexp<=5
bin2=(voi_data.yearsexp>5) & (voi_data.yearsexp<10)
bin3=voi_data.yearsexp>10
voi_data.yearsxpcats[bin1]='5less' # 5 years or less
voi_data.yearsxpcats[bin2]='5to10' #between 5 and 10 years
voi_data.yearsxpcats[bin3]='10plus' #10 or more years
dummies = pd.get_dummies(voi_data['yearsxpcats']).rename(columns=lambda x: 'yearsxpcats' + str(x))
df = pd.concat([voi_data, dummies], axis=1)
df = df[['education', 'sex', 'race', 'call', 'yearsxpcats10plus', 'yearsxpcats5less', 'yearsxpcats5to10']]

In [46]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
y=pd.DataFrame(df['call']).copy()
X=pd.DataFrame(data=df, columns=['education', 'sex', 'race', 'yearsxpcats10plus','yearsxpcats5less', 'yearsxpcats5to10']).copy()
X['intercept']=1

import statsmodels.api as sm
est = sm.Logit(y, X)
result = est.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.275993
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                   call   No. Observations:                 4870
Model:                          Logit   Df Residuals:                     4863
Method:                           MLE   Df Model:                            6
Date:                Sun, 09 Jun 2019   Pseudo R-squ.:                 0.01421
Time:                        11:05:22   Log-Likelihood:                -1344.1
converged:                       True   LL-Null:                       -1363.5
                                        LLR p-value:                 8.029e-07
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
education            -0.0699      0.198     -0.352      0.725      -0.459       0.319
sex     

Then I used backward selection methods to minimize to variables that showed significance within the model.

In [47]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
y=pd.DataFrame(df['call']).copy()
X=pd.DataFrame(data=df, columns=['race','yearsxpcats5less', 'yearsxpcats5to10']).copy()
X['intercept']=1

import statsmodels.api as sm
est = sm.Logit(y, X)
result = est.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.276052
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                   call   No. Observations:                 4870
Model:                          Logit   Df Residuals:                     4866
Method:                           MLE   Df Model:                            3
Date:                Sun, 09 Jun 2019   Pseudo R-squ.:                 0.01400
Time:                        11:05:23   Log-Likelihood:                -1344.4
converged:                       True   LL-Null:                       -1363.5
                                        LLR p-value:                 2.598e-08
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
race                -0.4412      0.108     -4.102      0.000      -0.652      -0.230
yearsxpcats

Write a story describing the statistical significance in the context or the original problem. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

**Conclusion: Despite a poor fitting regression model (r2), after controlling for sex, education, and years of experience; race, <5 years experience, and 5 - 10 years of experience, seems to have the strongest influence on receiving return calls. Based on the these explored estimates and from above race definitely has a significant impact on the number of return calls received. For future reference I would continue exploring possible confounding factors or spurious correlations between race and number of calls recieved.**